In [4]:
library("tidyverse")
library("dplyr")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [6]:
setwd('/home/ssark38/econ470/a0/work/hwk1')

In [9]:
# Month list --------------------------------------------------------------
y <- 2018
monthlist <- sprintf("%02d", 1:12)

In [10]:
# Reader ------------------------------------------------------------------
  read_service_area <- function(path) {
    read_csv(
      path, skip = 1,
      col_names = c(
        "contractid","org_name","org_type","plan_type","partial","eghp",
        "ssa","fips","county","state","notes"
      ),
      col_types = cols(
        contractid = col_character(),
        org_name   = col_character(),
        org_type   = col_character(),
        plan_type  = col_character(),
        partial    = col_logical(),
        eghp       = col_character(),
        ssa        = col_double(),
        fips       = col_double(),
        county     = col_character(),
        state      = col_character(),
        notes      = col_character()
      ),
      na = "*",
      show_col_types = FALSE,
      progress = FALSE
    )
  }

In [11]:
# One-month loader --------------------------------------------------------
  load_month_sa <- function(m, y) {
    path <- paste0("../ma-data/ma/service-area/Extracted Data/MA_Cnty_SA_",y, "_", m, ".csv")
    
    read_service_area(path) %>%
      mutate(month = as.integer(m), year = y)
  }

In [12]:
  # Read all months, then tidy once ----------------------------------------
  service.year <- map_dfr(monthlist, ~ load_month_sa(.x, y))

  # Ensure stable order before fills
  service.year <- service.year %>%
    arrange(contractid, fips, state, county, month)

  # Fill missing identifiers/labels
  service.year <- service.year %>%
    group_by(state, county) %>%
    fill(fips, .direction = "downup") %>%
    ungroup() %>%
    group_by(contractid) %>%
    fill(plan_type, partial, eghp, org_type, org_name, .direction = "downup") %>%
    ungroup()

  # Collapse to yearly: one row per contract × county (fips) × year --------
  final.service.area <- service.year %>%
    group_by(contractid, fips, year) %>%
    arrange(month, .by_group = TRUE) %>%
    summarize(
      state     = last(state),
      county    = last(county),
      org_name  = last(org_name),
      org_type  = last(org_type),
      plan_type = last(plan_type),
      partial   = last(partial),
      eghp      = last(eghp),
      ssa       = last(ssa),
      notes     = last(notes),
      .groups = "drop"
    )

Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”
Warning message:
“One or mo

In [13]:
write_csv(final.service.area, "data/service_data.csv")

In [7]:
service_data <- read_csv("data/service_data.csv")

Rows: 331593 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (7): contractid, state, county, org_name, org_type, plan_type, notes
dbl (3): fips, year, ssa
lgl (2): partial, eghp

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [8]:
head(service_data)

contractid,fips,year,state,county,org_name,org_type,plan_type,partial,eghp,ssa,notes
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<dbl>,<chr>
90091,NA,2018,NA,NA,UNITED MINE WORKERS OF AMERICA HLTH & RETIREMENT,HCPP - 1833 Cost,HCPP - 1833 Cost,NA,NA,NA,"Covers the entire US, all States and Counties"
H0022,39023,2018,OH,Clark,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Demo,Medicare-Medicaid Plan HMO/HMOPOS,NA,NA,36110,NA
H0022,39035,2018,OH,Cuyahoga,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Demo,Medicare-Medicaid Plan HMO/HMOPOS,NA,NA,36170,NA
H0022,39051,2018,OH,Fulton,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Demo,Medicare-Medicaid Plan HMO/HMOPOS,NA,NA,36260,NA
H0022,39055,2018,OH,Geauga,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Demo,Medicare-Medicaid Plan HMO/HMOPOS,NA,NA,36280,NA
H0022,39057,2018,OH,Greene,"BUCKEYE COMMUNITY HEALTH PLAN, INC.",Demo,Medicare-Medicaid Plan HMO/HMOPOS,NA,NA,36290,NA
